In [1]:
import jpy_tools.parseSnake as jps

In [2]:
snakeFile = jps.SnakeMakeFile()

In [12]:
snakeHeader = jps.SnakeHeader(snakeFile, '/public/home/liuzj/scripts/pipeline/polyACallerJb/snakemake/config.yaml')
snakeHeader.generateContent()

config contents:

    pipelineDir     :/public/home/liuzj/scripts/pipeline/polyACallerJb/script/

     resultDir      :/public/home/liuzj/scripts/pipeline/polyACallerJb/results/

     nanoporeFa     :/public/home/liuzj/scripts/pipeline/polyACallerJb/data/nanopore.fa

   nanoporeBasSum   :/public/home/liuzj/scripts/pipeline/polyACallerJb/data/nanoporeSeqSummary.txt

 nanoporeWorkSpace  :/public/home/liuzj/scripts/pipeline/polyACallerJb/data/workspace/

      genomeFa      :/public/home/liuzj/data/Araport11/genome.fa



In [13]:
mappingGenome = jps.SnakeRule(snakeFile, 'mappingGenome', 1, 56)
mappingGenome.setInput(b=['nanoporeFa'])
mappingGenome.setOutput(a=['mappingBam.bam'])
mappingGenome.setParams(b=['genomeFa'])
mappingGenome.setShell("""
minimap2 -t {threads} -ax splice --secondary=no -G 12000 {params.genomeFa} {input.nanoporeFa} | samtools sort -@ {threads} -o {output.mappingBam} -
samtools index -@ {threads} {output.mappingBam}
""")
mappingGenome.generateContent()

In [19]:
findAdapter = jps.SnakeRule(snakeFile, 'findAdapter', 2, 56)
findAdapter.setInput(b=['nanoporeFa'], a=['mappingBam.bam'])
findAdapter.setOutput(a=['adapterResult.tsv'])
findAdapter.setShell("""
python adapterFinder.py --inbam {input.mappingBam} --inseq {input.nanoporeFa} --out {output.adapterResult} --threads {threads}
""")
findAdapter.generateContent()

In [20]:
polyacaller = jps.SnakeRule(snakeFile, 'polyacaller', 3, 56)
polyacaller.setInput(a=['adapterResult.tsv'], b=['nanoporeSum', 'nanoporeWorkSpace'])
polyacaller.setOutput(a=['polyATailResult.tsv'])
polyacaller.setShell("""
python PolyACaller.py --inadapter {input.adapterResult} --summary {input.nanoporeSum}  --fast5dir {input.nanoporeWorkSpace} --out {output.polyATailResult} --threads {threads}
""")
polyacaller.generateContent()

In [21]:
snakeAll = jps.SnakeAll(snakeFile)
snakeAll.generateContent(polyATailResult=0)

In [18]:
snakeFile.generateContent('./snakefile')

configfile: "/public/home/liuzj/scripts/pipeline/polyACallerJb/snakemake/config.yaml"
pipelineDir = config['pipelineDir']


rule all:
    input:
        polyATailResult = f"{config['resultDir']}step3_polyacaller/polyATailResult.tsv"

rule mappingGenome:
    input:
        nanoporeFa = config['nanoporeFa']
    output:
        mappingBam = f"{config['resultDir']}step1_mappingGenome/mappingBam.bam"
    params:
        genomeFa = config['genomeFa'],
        gpu = "0"

    threads:56
    shell:
        """
cd {pipelineDir}
minimap2 -t {threads} -ax splice --secondary=no -G 12000 {params.genomeFa} {input.nanoporeFa} | samtools sort -@ {threads} -o {output.mappingBam} -
samtools index -@ {threads} {output.mappingBam}
        """

rule findAdapter:
    input:
        nanoporeFa = config['nanoporeFa'],
        mappingBam = f"{config['resultDir']}step1_mappingGenome/mappingBam.bam"
    output:
        adapterResult = f"{config['resultDir']}step2_findAdapter/adapterResult.tsv"
    params:
       